In [76]:
import numpy as np
import scipy as sp
from algorithms import QuboTriU, get_A, greedy
import joblib
import networkx as nx
import utils
import json
from glob import glob 
import os
import pandas as pd

In [84]:
df_optimum = pd.read_html("solutions.html")[0]
df_optimum.drop(df_optimum.index[0], inplace=True)
df_optimum.columns = [40, '40_optimum', 80, '80_optimum', 100, '100_optimum',
                120, '120_optimum', 140, '140_optimum', 160, '160_optimum']
df_optimum

,40,40_optimum,80,80_optimum,100,100_optimum,120,120_optimum,140,140_optimum,160,160_optimum
1,kcluster40_025_10_1,29,kcluster80_025_20_1,94,kcluster100_025_25_1,139,kcluster120_025_30_1,198,kcluster140_025_35_1,259,kcluster160_025_40_1,338
2,kcluster40_025_10_2,30,kcluster80_025_20_2,93,kcluster100_025_25_2,138,kcluster120_025_30_2,199,kcluster140_025_35_2,263,kcluster160_025_40_2,338
3,kcluster40_025_10_3,27,kcluster80_025_20_3,100,kcluster100_025_25_3,142,kcluster120_025_30_3,197,kcluster140_025_35_3,257,kcluster160_025_40_3,333
4,kcluster40_025_10_4,27,kcluster80_025_20_4,96,kcluster100_025_25_4,140,kcluster120_025_30_4,198,kcluster140_025_35_4,261,kcluster160_025_40_4,333
5,kcluster40_025_10_5,26,kcluster80_025_20_5,97,kcluster100_025_25_5,145,kcluster120_025_30_5,199,kcluster140_025_35_5,266,kcluster160_025_40_5,328
6,kcluster40_025_20_1,77,kcluster80_025_40_1,280,kcluster100_025_50_1,417,kcluster120_025_60_1,600,kcluster140_025_70_1,806,kcluster160_025_80_1,1047
7,kcluster40_025_20_2,84,kcluster80_025_40_2,273,kcluster100_025_50_2,417,kcluster120_025_60_2,611,kcluster140_025_70_2,805,kcluster160_025_80_2,1042
8,kcluster40_025_20_3,76,kcluster80_025_40_3,285,kcluster100_025_50_3,429,kcluster120_025_60_3,598,kcluster140_025_70_3,792,kcluster160_025_80_3,1026
9,kcluster40_025_20_4,75,kcluster80_025_40_4,284,kcluster100_025_50_4,413,kcluster120_025_60_4,600,kcluster140_025_70_4,795,kcluster160_025_80_4,1033
10,kcluster40_025_20_5,73,kcluster80_025_40_5,292,kcluster100_025_50_5,435,kcluster120_025_60_5,604,kcluster140_025_70_5,816,kcluster160_025_80_5,1021


We are generating qubos for the benchmark data to calculate the DkS using the quadratic penalty approach
 - g = (read A)
 - g = complement(g)  (because, we work with SkS)
 - $\frac{\mu}{2} > \min(\tilde{m}, k-1)$   where $\tilde{m}$ is no. of edges with heuristic (Greedy)
 - $\mu = 2\min(\tilde{m}, k-1) + 1$
 - Q = $\frac{1}{2} x^T A x + \frac{\mu}{2} (e^T x - k)^2$
 - Since BiqBin only works with Q containing integer elements, we consider Q = 2Q
  

In [128]:
def get_nearby_mus(mu):
    mus = []
    for i in range(10, -11, -2):
        if (mu+i <= 0):
            break
        mus.append(mu+i)
    return mus

In [152]:
sizes = [40]

for size in sizes:

    qubo_folder = f"qubo_folder/{size}"
    os.makedirs(qubo_folder, exist_ok=True)

    input_data_folder = f"k-cluster_{size}"
    files = os.listdir(input_data_folder)
    files = sorted([file for file in files if "_1.dat" in file])

    for file in files:
        instance_name = file.removesuffix(".dat")
        opt_val = int(df_optimum[df_optimum[size] == instance_name][f'{size}_optimum'].values[0])

        A_raw, A, k = get_A(f"{input_data_folder}/{file}")
        g = nx.from_numpy_array(A)
        g = nx.complement(g)

        g_greedy, dmax, dvag = greedy(g=g, k=k)
        mu_lb = 2*min(g_greedy.number_of_edges(), k-1) + 1
        mu_lb = int(mu_lb)
        
        print(f"{mu_lb = }")

        mus = get_nearby_mus(mu=mu_lb)

        os.makedirs(f"{qubo_folder}/{instance_name}", exist_ok=True)

        for mu in mus:
            qubo_obj = QuboTriU(g=g)
            Q = qubo_obj.update(l=0, mu = mu, k=k)
            Q = 2*np.array(Q)
            data = utils.qubo_to_biqbin_representation(Q)
            data["instance"] = instance_name
            data["mu_lb"] = mu_lb
            data["mu"] = mu
            data["optimum"] = opt_val

            qubo_file_name = f"{instance_name}_mulb_{mu_lb}_mu_{mu}.json"
            with open(f"{qubo_folder}/{instance_name}/{qubo_file_name}", "w") as f:
                json.dump(data, f)


mu_lb = 19
mu_lb = 39
mu_lb = 59
mu_lb = 15
mu_lb = 39
mu_lb = 59
mu_lb = 1
mu_lb = 39
mu_lb = 59


In [153]:
sorted(os.listdir("qubo_folder/40"))

['kcluster40_025_10_1',
 'kcluster40_025_20_1',
 'kcluster40_025_30_1',
 'kcluster40_050_10_1',
 'kcluster40_050_20_1',
 'kcluster40_050_30_1',
 'kcluster40_075_10_1',
 'kcluster40_075_20_1',
 'kcluster40_075_30_1']

In [156]:
sizes = [40]

for size in sizes:
    for instance_name in os.listdir(f"qubo_folder/{size}"):
        output_folder = f"outputs/{size}/{instance_name}/"
        os.makedirs(output_folder, exist_ok=True)
        for file in glob(f"qubo_folder/{size}/{instance_name}/*"):

            print(f"mpirun python3 biqbin_qubo.py {file} params {output_folder}")

mpirun python3 biqbin_qubo.py qubo_folder/40/kcluster40_050_20_1/kcluster40_050_20_1_mulb_39_mu_37.json params outputs/40/kcluster40_050_20_1/
mpirun python3 biqbin_qubo.py qubo_folder/40/kcluster40_050_20_1/kcluster40_050_20_1_mulb_39_mu_35.json params outputs/40/kcluster40_050_20_1/
mpirun python3 biqbin_qubo.py qubo_folder/40/kcluster40_050_20_1/kcluster40_050_20_1_mulb_39_mu_29.json params outputs/40/kcluster40_050_20_1/
mpirun python3 biqbin_qubo.py qubo_folder/40/kcluster40_050_20_1/kcluster40_050_20_1_mulb_39_mu_33.json params outputs/40/kcluster40_050_20_1/
mpirun python3 biqbin_qubo.py qubo_folder/40/kcluster40_050_20_1/kcluster40_050_20_1_mulb_39_mu_45.json params outputs/40/kcluster40_050_20_1/
mpirun python3 biqbin_qubo.py qubo_folder/40/kcluster40_050_20_1/kcluster40_050_20_1_mulb_39_mu_49.json params outputs/40/kcluster40_050_20_1/
mpirun python3 biqbin_qubo.py qubo_folder/40/kcluster40_050_20_1/kcluster40_050_20_1_mulb_39_mu_39.json params outputs/40/kcluster40_050_20_1/

In [28]:
data = joblib.load("SAAlgoLinear")
# data = joblib.load("SAAlgoQuadraticPenalty")
data

{'kcluster40_025_10_1': {'successful': True,
  'optimum': 29,
  'target_cardinality_reached': np.True_,
  'edges_greedy': 28,
  'edges_algo': 29,
  'summary': {'g': <networkx.classes.graph.Graph at 0x75670047c260>,
   'sampler_response': {'x': array([1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1]),
    'value': np.float64(-34.0),
    'cardinality': np.int64(10),
    'sampler_response': SampleSet(rec.array([([1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1], -34., 1),
               ([1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1], -34., 1),
               ([1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1], -34., 1),
               ([1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [66]:
qubo_object = data["kcluster40_025_10_1"]['summary']["additional_info"]["algo_summary"]["qubo"]
qubo_object

In [67]:
data['kcluster40_025_10_1']['summary']["additional_info"]["algo_summary"]['mui']

0

In [68]:
# qubo_object.update(l=0, mu=5, k=10)

In [69]:
qubo = qubo_object.Q
qubo = 2*qubo
qubo

array([[-95.,  12.,  12., ...,  10.,  10.,  10.],
       [  0., -95.,  12., ...,  12.,  12.,  10.],
       [  0.,   0., -95., ...,  10.,  10.,  12.],
       ...,
       [  0.,   0.,   0., ..., -95.,  10.,  10.],
       [  0.,   0.,   0., ...,   0., -95.,  12.],
       [  0.,   0.,   0., ...,   0.,   0., -95.]], shape=(40, 40))

In [70]:
q_int = np.array(qubo, dtype=int)
q_int
np.all(q_int == qubo)
    

np.True_

In [71]:
import json

qubo_json = {"qubo": to_sparse(qubo)}
with open("qubo_example_2.json", "w") as f:
    json.dump(qubo_json, f)

In [48]:
with open ("qubo_example_2.json.output.json", "r") as f:
    res = json.load(f)
res['qubo']['cardinality']

10.0